# High-level Keras R (TF) RNN Example

In [1]:
# SETUP
#
# Install keras R
# install.packages('keras', repos = "https://cloud.r-project.org")
# 
# Update reticulate from cran (it defaults to mran which has an outdated version)
# install.packages("reticulate", repos = "https://cloud.r-project.org")

In [2]:
library(keras)
use_python('/anaconda/envs/py35')

# Import util functions
source("./common/utils.R")

# Import hyper-parameters
params <- load_params('lstm')

Loading required package: rjson


In [3]:
# reticulate::py_config()

In [4]:
cat("OS:", Sys.info()["sysname"], "\n")
cat(R.version$version.string, "\n")
cat("Keras:", paste0(packageVersion("keras")), "\n")
cat("Tensorflow:", paste0(packageVersion("tensorflow")), "\n")
cat("Keras using", backend()$backend(), "\n")
cat("Keras channel ordering is", backend()$image_data_format(), "\n") 
cat("GPU: ", get_gpu_name(), "\n")
cat(get_cuda_version(), "\n")
cat(get_cudnn_version(), "\n")

OS: Linux 
R version 3.4.1 (2017-06-30) 
Keras: 2.1.5 
Tensorflow: 1.5 
Keras using tensorflow 
Keras channel ordering is channels_last 
GPU:  Tesla P100-PCIE-16GB 
CUDA Version 8.0.61 
CuDNN Version 6.0.21 


In [5]:
create_symbol <- function(CUDNN = TRUE, maxf = params$MAXFEATURES, edim = params$EMBEDSIZE, nhid = params$NUMHIDDEN, maxl = params$MAXLEN){
    
    model <- keras_model_sequential() %>%
    
    layer_embedding(maxf, edim, input_length = maxl)
    
    if (CUDNN){
        model %>% layer_cudnn_gru(units = nhid, return_sequences = FALSE, return_state = FALSE)
        } else{
        model %>% layer_gru(units = nhid, return_sequences = FALSE, return_state = FALSE)    
    }
    
    model %>% layer_dense(2, activation = "softmax")
    
    return(model)
}

In [6]:
init_model <- function(m, lr=params$LR, b1=params$BETA_1, b2=params$BETA_2, eps=params$EPS){
    m %>% compile(
      loss = "categorical_crossentropy",
      optimizer = optimizer_adam(lr = lr, beta_1 = b1, beta_2 = b2, epsilon = eps),
      metrics = "accuracy"
    )
    return(m)
}

In [7]:
imdb <- imdb_for_library()
x_train <- imdb$x_train
y_train <- imdb$y_train
x_test <- imdb$x_test
y_test <- imdb$y_test
rm(imdb)

y_train <- to_categorical(y_train, num_classes = 2)
y_test <- to_categorical(y_test, num_classes = 2)

In [8]:
cat('x_train shape:', dim(x_train), '\n')
cat('x_test shape:', dim(x_test), '\n')
cat('y_train shape:', dim(y_train), '\n')
cat('y_test shape:', dim(y_test), '\n')

x_train shape: 25000 150 
x_test shape: 25000 150 
y_train shape: 25000 2 
y_test shape: 25000 2 


In [9]:
# Load symbol
sym = create_symbol(CUDNN = TRUE)

In [10]:
# Initialise model
model = init_model(sym)

In [11]:
summary(model)

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
embedding_1 (Embedding)             (None, 150, 125)                3750000     
________________________________________________________________________________
cu_dnngru_1 (CuDNNGRU)              (None, 100)                     68100       
________________________________________________________________________________
dense_1 (Dense)                     (None, 2)                       202         
Total params: 3,818,302
Trainable params: 3,818,302
Non-trainable params: 0
________________________________________________________________________________


In [12]:
# Main training loop
system.time(
    model %>% fit(x_train,
          y_train,
          batch_size=params$BATCHSIZE,
          epochs=params$EPOCHS,
          verbose=1)
)

   user  system elapsed 
 21.399   4.443  25.228 

In [13]:
# Main evaluation loop
y_guess <- model %>% predict_classes(x_test, batch_size = params$BATCHSIZE)
y_truth <- apply(y_test, 1, function(x) which.max(x)-1)

In [14]:
print(paste0("Accuracy: ", sum(y_guess == y_truth)/length(y_guess)))

[1] "Accuracy: 0.85124"
